#1. 安裝套件

In [ ]:
!pip install Beautifulsoup4
!pip install requests
!pip install pandas

# 2.套件與參數設定

目標網站: https://www.ptt.cc/bbs/Tech_Job/index.html

上一頁: https://www.ptt.cc/bbs/Tech_Job/index4002.html

上上一頁: https://www.ptt.cc/bbs/Tech_Job/index4001.html

In [ ]:
# 目標網址
target_url = "https://www.ptt.cc/bbs/"

# 目標看板
target_board = "Tech_Job"

#目標頁面
target_page = "/index"

#目標頁數
page_num = ""

#頁面附屬檔名
page_ext = ".html"

target = target_url + target_board + target_page + page_num  + page_ext

準備 requests 用的 header

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
}

print(headers)

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}


# 3. 用 Requests 下載目標網頁的程式碼

In [ ]:
import requests
data = requests.get(target, headers= headers)
print(data.content)

b'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>\xe7\x9c\x8b\xe6\x9d\xbf Tech_Job \xe6\x96\x87\xe7\xab\xa0\xe5\x88\x97\xe8\xa1\xa8 - \xe6\x89\xb9\xe8\xb8\xa2\xe8\xb8\xa2\xe5\xaf\xa6\xe6\xa5\xad\xe5\x9d\x8a</title>\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">\n\n\n\n\n\t</head>\n    <body>\n\t\t\n<div id="topbar-container">\n\t<div id="topbar" class="bbs-content">\n\t\t<a id="logo" href="/bbs/">\xe6\x89\xb9\xe8\xb8\xa2\xe8\xb8\xa2\xe5\xaf\xa6\xe6\xa5

# 4. 用 BeautifulSoup 分析跟取得 html 標籤資料

In [ ]:
from bs4 import BeautifulSoup
html_code = BeautifulSoup(data.content)

In [ ]:
div_list = html_code.find_all('div', class_="title")

'''
從 <div> list 裡面元素找尋其他標籤的範例
並從目標標籤取的 html 屬性值 與 標籤文字
a_tag = div_list[0].find('a')
a_url = a_tag.attrs['href']
a_title = a_tag.contents[0]
'''

urls= []
for div_ in div_list:
  try:
    a_tag = div_.find('a')
    #print(a_tag.attrs['href'], a_tag.contents[0])

    url_ = {
    'url': a_tag.attrs['href'],
    'title': a_tag.contents[0]
    }

  except:
    url_ = {
    'url':  None,
    'title': '文章已刪除'
    }
  urls.append(url_)

print(urls[0]['url'])
print(urls[0]['title'])
print(urls[1]['url'])
print(urls[1]['title'])

/bbs/Tech_Job/M.1723723186.A.404.html
[請益]offer 請益
/bbs/Tech_Job/M.1723726637.A.91F.html
[新聞] 公告了！台積電以171.4億元買群創南科四廠


# 5. For Loop 針對每篇文章連結進行讀取

In [ ]:
#for loop
import time

ptt_data = []
for url_ in urls:
  if url_['url'] != None:
    article_url = 'https://www.ptt.cc' + url_['url']
    page_data = requests.get(article_url, headers=headers)
    page_html_code = BeautifulSoup(page_data.content)
    article_data = page_html_code.find_all('span', class_="article-meta-value")
    article_author = article_data[0].contents[0]
    article_title = article_data[2].contents[0]
    article_time = article_data[3].contents[0]
    article_body = page_html_code.find('div', id='main-content').contents[4]
    article_row = {
        'url': article_url,
        'title': article_title,
        'author': article_author,
        'time': article_time,
        'content': article_body
    }
    ptt_data.append(article_row)
  time.sleep(1)


# 6. 讀取所有文章並保存

In [ ]:
import pandas
ptt_df = pandas.DataFrame(ptt_data)
# ....

ptt_df.to_csv()

',url,title,author,time,content\n0,https://www.ptt.cc/bbs/Tech_Job/M.1723723186.A.404.html,[請益]offer 請益,s19273 (阿翔),Thu Aug 15 19:59:44 2024,"\n（朋友沒有帳號代po）\n\n大家好，本人有廠務十年經驗  因工作內容與主管描述不相符\n 加上個人因素  故已離職 現在已面試多家  剩下兩家目前在考慮中\n想請各位大大給點意見\n\nN=上份工作底薪\n1.\n公司：力積電\n職級：廠務輪班工程師\n地點：竹科\n薪水：(N+13)*14 季獎金  分紅1~2\n加班費：有\n工時：12 做二休二\n\n\n2.\n公司：緯創\n職級：廠務工程師\n地點：竹科\n薪水：(N+5k)*14 分紅2~4\n加班費：有\n工時：常日班 因應產線擴編 之後可能需要輪班\n\n力積電相對來說較穩定 只是沒有賺錢分紅較低  也比較沒有向上發展的機會\n\n緯創目前再擴廠 工作內容較為繁雜  但分紅比較高  也有比較多發展及學習空間\n\n想請版上大大給予相關意見  感謝\n\n感謝各位前輩回覆提供意見\n\n--\n"\n1,https://www.ptt.cc/bbs/Tech_Job/M.1723726637.A.91F.html,[新聞] 公告了！台積電以171.4億元買群創南科四廠,hvariables (Speculative Male),Thu Aug 15 20:57:06 2024,"\n"\n2,https://www.ptt.cc/bbs/Tech_Job/M.1723736590.A.00B.html,Re: [請益] 日揚科技-研發製程,h34143202 (神犬豆豆),Thu Aug 15 23:43:08 2024,"\n哈嘍，我本週有收到這家邀約，請問您有去面試嗎？\n\n想詢問一下面試內容。\n\n--\n"\n3,https://www.ptt.cc/bbs/Tech_Job/M.1723767127.A.537.html,[新聞] 軟銀與英特爾合作破局\u3000外媒曝轉向尋求和,jeff0025 (無法顯示人物名稱),Fri Aug 16 08:12:03 2024,"\n\n\nETtoday財經雲 

In [ ]:
article_url = 'https://www.ptt.cc' + urls[0]['url']
article_url

# request 下載資料
page_data = requests.get(article_url, headers=headers)

# 解析 Requeset 結果
page_html_code = BeautifulSoup(page_data.content)

#找到文章 meta info
article_data = page_html_code.find_all('span', class_="article-meta-value")

article_author = article_data[0].contents[0]

article_title = article_data[2].contents[0]

article_time = article_data[3].contents[0]

#文章內文
article_body = page_html_code.find('div', id='main-content').contents[4]

article_row = {
    'url': article_url,
    'title': article_title,
    'author': article_author,
    'time': article_time,
    'content': article_body
}

print(article_row)

{'url': 'https://www.ptt.cc/bbs/Tech_Job/M.1723723186.A.404.html', 'title': '[請益]offer 請益', 'author': 's19273 (阿翔)', 'time': 'Thu Aug 15 19:59:44 2024', 'content': '\n（朋友沒有帳號代po）\n\n大家好，本人有廠務十年經驗  因工作內容與主管描述不相符\n 加上個人因素  故已離職 現在已面試多家  剩下兩家目前在考慮中\n想請各位大大給點意見\n\nN=上份工作底薪\n1.\n公司：力積電\n職級：廠務輪班工程師\n地點：竹科\n薪水：(N+13)*14 季獎金  分紅1~2\n加班費：有\n工時：12 做二休二\n\n\n2.\n公司：緯創\n職級：廠務工程師\n地點：竹科\n薪水：(N+5k)*14 分紅2~4\n加班費：有\n工時：常日班 因應產線擴編 之後可能需要輪班\n\n力積電相對來說較穩定 只是沒有賺錢分紅較低  也比較沒有向上發展的機會\n\n緯創目前再擴廠 工作內容較為繁雜  但分紅比較高  也有比較多發展及學習空間\n\n想請版上大大給予相關意見  感謝\n\n感謝各位前輩回覆提供意見\n\n--\n'}


# 7. 儲存到 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
ptt_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Course/ppt.csv')



---

# 第二部分 API 撈取資料
# 1. 準備參數


In [ ]:
# URL
stock_query_url = 'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={}&stockNo={}&response={}&_={}'

#查詢參數
query_date = '20240816'
stockNo = '2330'
response_ = 'json'
timestamp_ = '1723772411167'

# Requests 參數
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
}

# 合併參數與URL
stock_param_url = stock_query_url.format(query_date, stockNo, response_, timestamp_)

print(stock_param_url)

https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240816&stockNo=2330&response=json&_=1723772411167


#2. 用 Requests 取得API回應

In [ ]:
import requests

res = requests.get(stock_param_url, verify=False)
res.text

'{"stat":"OK","date":"20240816","title":"113年08月 2330 台積電           各日成交資訊","fields":["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"],"data":[["113/08/01","50,257,642","48,211,390,158","969.00","969.00","948.00","960.00","+26.00","66,845"],["113/08/02","89,044,852","81,488,499,047","918.00","928.00","903.00","903.00","-57.00","451,562"],["113/08/05","159,662,795","133,482,646,539","852.00","857.00","813.00","815.00","-88.00","588,373"],["113/08/06","108,521,787","94,271,532,567","868.00","895.00","840.00","880.00","+65.00","189,677"],["113/08/07","62,643,163","56,900,026,373","901.00","920.00","893.00","920.00","+40.00","109,333"],["113/08/08","41,288,222","37,045,507,217","901.00","905.00","887.00","896.00","-24.00","107,303"],["113/08/09","54,464,183","50,586,946,042","927.00","934.00","922.00","934.00","+38.00","94,661"],["113/08/12","40,968,742","38,572,439,136","942.00","948.00","931.00","940.00","+6.00","73,731"],["113/08/13","26,831,364","25,227,760,336","945.00","945.

#3 . 用 JSON 格式化輸出

In [ ]:
import json
import pandas
stock_data = json.loads(res.text)

print(json.dumps(stock_data, sort_keys=False, indent=4, separators=(',', ':'), ensure_ascii=False))

{
    "stat":"OK",
    "date":"20240816",
    "title":"113年08月 2330 台積電           各日成交資訊",
    "fields":[
        "日期",
        "成交股數",
        "成交金額",
        "開盤價",
        "最高價",
        "最低價",
        "收盤價",
        "漲跌價差",
        "成交筆數"
    ],
    "data":[
        [
            "113/08/01",
            "50,257,642",
            "48,211,390,158",
            "969.00",
            "969.00",
            "948.00",
            "960.00",
            "+26.00",
            "66,845"
        ],
        [
            "113/08/02",
            "89,044,852",
            "81,488,499,047",
            "918.00",
            "928.00",
            "903.00",
            "903.00",
            "-57.00",
            "451,562"
        ],
        [
            "113/08/05",
            "159,662,795",
            "133,482,646,539",
            "852.00",
            "857.00",
            "813.00",
            "815.00",
            "-88.00",
            "588,373"
        ],
        [
            "113/08/06",

# 4. 用 Pasdas 載入資料與欄位名稱

In [ ]:
import pandas
stock_df = pandas.DataFrame(stock_data['data'], columns=stock_data['fields'])
stock_df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/08/01,"50,257,642","48,211,390,158",969.00,969.00,948.00,960.00,+26.00,"66,845"
1,113/08/02,"89,044,852","81,488,499,047",918.00,928.00,903.00,903.00,-57.00,"451,562"
2,113/08/05,"159,662,795","133,482,646,539",852.00,857.00,813.00,815.00,-88.00,"588,373"
3,113/08/06,"108,521,787","94,271,532,567",868.00,895.00,840.00,880.00,+65.00,"189,677"
4,113/08/07,"62,643,163","56,900,026,373",901.00,920.00,893.00,920.00,+40.00,"109,333"
5,113/08/08,"41,288,222","37,045,507,217",901.00,905.00,887.00,896.00,-24.00,"107,303"
6,113/08/09,"54,464,183","50,586,946,042",927.00,934.00,922.00,934.00,+38.00,"94,661"
7,113/08/12,"40,968,742","38,572,439,136",942.00,948.00,931.00,940.00,+6.00,"73,731"
8,113/08/13,"26,831,364","25,227,760,336",945.00,945.00,937.00,941.00,+1.00,"39,633"
9,113/08/14,"40,527,779","38,507,151,446",951.00,955.00,945.00,948.00,+7.00,"56,190"


In [ ]:
# 計算平均值
pandas.to_numeric(stock_df['開盤價']).mean()

# 計算平均值: 資料帶有格式化
pandas.to_numeric(stock_df['成交股數'].str.replace(',','').astype(int)).mean()

919.9090909090909

# 5. 機器學習實現案例 (以 LinearRegression 為例)

In [ ]:
# 安裝套件
!pip install scikit-learn

# 參數保存套件
!pip install joblib

In [ ]:
# 建立訓練資料 (x)
x = stock_df.iloc[:, 3].values

x = stock_df.iloc[:, 3].drop(10, inplace=False, axis=0).values

x = stock_df.iloc[:, 3].drop(10, inplace=False, axis=0).values.reshape(-1, 1)

x

array([['969.00'],
       ['918.00'],
       ['852.00'],
       ['868.00'],
       ['901.00'],
       ['901.00'],
       ['927.00'],
       ['942.00'],
       ['945.00'],
       ['951.00']], dtype=object)

In [ ]:
# 建立Label (Y)
y = stock_df.iloc[:,6].values

y=stock_df.iloc[:, 6].drop(0, inplace=False, axis=0).values

array(['903.00', '815.00', '880.00', '920.00', '896.00', '934.00',
       '940.00', '941.00', '948.00', '943.00'], dtype=object)

In [ ]:
# 導入 sklearn 的線性回歸模型物件
from sklearn.linear_model import LinearRegression

# 初始化 LR Model
lr = LinearRegression()

# 用資料學習模型
lr.fit(x,y)

# ----

# 測試資料建立
x_test = [
    [500],
    [100],
    [200]
]

# 用學習完的模型進行預測
y_test = lr.predict(x_test)

# 印出預測結果
y_test

array([761.83012938, 620.69303294, 655.97730705])

# 6. 模型保存與使用

In [ ]:
# 保存模型 (方法一)
from google.colab import drive
import joblib

# 掛載雲端硬碟
drive.mount('/content/gdrive')

# 將整個物件保存成檔案
joblib.dump(lr, '/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816')

['/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816']

In [ ]:
# 保存模型 (方法二)
import pickle

filename = '/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
from sklearn.linear_model import LinearRegression
from time import time
import pickle
import _pickle as cPickle
import joblib
import os

print("Joblib 載入 Pickle 檔案:")
t1 = time()
lr_model_lib = joblib.load('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl')
print("Joblib module 載入時間:", time() - t1)
print("Joblib module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl'), "kB \n")

print("Joblib 載入 Joblib 檔案:")
t1 = time()
lr_model_lib = joblib.load('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816')
print("Joblib module 載入時間:", time() - t1)
print("Joblib module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816'), "kB \n")

print("==================================\n")

# print("Pickle 載入 Joblib 檔案:\n")
# t1 = time()
# lr_model_pkl =  pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816', 'rb'))
# print("Pickle module 載入時間:", time() - t1)
# print("Pickle module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816'), "kB \n")

print("Pickle 載入 Pickle 檔案:")
t1 = time()
lr_model_pkl =  pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl', 'rb'))
print("Pickle module 載入時間:", time() - t1)
print("Pickle module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl'), "kB \n")
print("==================================\n")

# print("cPickle 載入 Joblib 檔案:\n")
# t1 = time()
# lr_model_pkl =  cPickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816', 'rb'))
# print("Pickle module 載入時間:", time() - t1)
# print("Pickle module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816'), "kB \n")
# print(time() - t1)

print("cPickle 載入 Pickle 檔案:")
t1 = time()
lr_model_pkl =  cPickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl', 'rb'))
print("cPickle module 載入時間:", time() - t1)
print("cPickle module file size", os.path.getsize('/content/gdrive/My Drive/Colab Notebooks/Course/lr_model2024816.pkl'), "kB \n")



Joblib 載入 Pickle 檔案:
Joblib module 載入時間: 0.008245229721069336
Joblib module file size 419 kB 

Joblib 載入 Joblib 檔案:
Joblib module 載入時間: 0.00749516487121582
Joblib module file size 560 kB 


Pickle 載入 Pickle 檔案:
Pickle module 載入時間: 0.005443572998046875
Pickle module file size 419 kB 


cPickle 載入 Pickle 檔案:
cPickle module 載入時間: 0.01509714126586914
cPickle module file size 419 kB 



# 7. Line Notify

In [ ]:
import requests
import time
import json

# 定義參數
line_notify_url = "https://notify-api.line.me/api/notify"
line_notify_token = "YOUR TOKEN"

# Request Header => Postman 的 Authorization
line_notify_header = {
    'Authorization': 'Bearer {}'.format(line_notify_token)
}

# Request Body => Postman 的 Params
# 這邊可以自訂義你的傳輸訊息或訊息樣板
line_notify_body = {
    'message':  "傳輸訊息"
}

res = requests.post(line_notify_url, headers=line_notify_header, data = line_notify_body)
res_msg =  json.loads(res.text)
print("[{}] {}".format(res_msg['status'], res_msg['message']))


[200] ok
